In [1]:
import requests 
from bs4 import BeautifulSoup 
import re
import pandas as pd
import numpy as np

In [2]:
a = 'http://www.taipeibo.com/year/2017'
response = requests.get(a)
response.encoding

'ISO-8859-1'

In [3]:
# 一定要編碼
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, "lxml") 
yearlist =[i for i in soup.table.find_all("tr")]
yearlist

[<tr class="tb-top">
 <th>排名</th>
 <th>中文片名</th>
 <th>英文片名</th>
 <th>院數</th>
 <th>映期</th>
 <th>上映日期</th>
 <th>平均票房</th>
 <th class="import">累積票房</th>
 </tr>, <tr>
 <td class="list-r8 o-n" data-th="排名">1</td>
 <td class="list-r1" data-th="中文片名">玩命關頭8</td>
 <td class="list-r2" data-th="英文片名">Fast &amp; Furious 8</td>
 <td class="list-r3" data-th="院數">25</td>
 <td class="list-r4" data-th="映期">85</td>
 <td class="list-r5" data-th="上映日期">2017/04/12</td>
 <td class="list-r6" data-th="平均票房">7,341,639</td>
 <td class="import list-r7" data-th="累積票房">183,540,980</td>
 </tr>, <tr>
 <td class="list-r8 o-n" data-th="排名">2</td>
 <td class="list-r1" data-th="中文片名">與神同行</td>
 <td class="list-r2" data-th="英文片名">Along with the Gods: The Two Worlds</td>
 <td class="list-r3" data-th="院數">24</td>
 <td class="list-r4" data-th="映期">60</td>
 <td class="list-r5" data-th="上映日期">2017/12/22</td>
 <td class="list-r6" data-th="平均票房">7,075,516</td>
 <td class="import list-r7" data-th="累積票房">169,812,385</td>
 </tr>, 

In [4]:
# These strings tend to have a lot of extra whitespace, which you can remove by using the .stripped_strings generator instead
# 為 generator 所組成，要用FOR去PRINT
column_name = [i for i in yearlist[0].stripped_strings]
column_name

['排名', '中文片名', '英文片名', '院數', '映期', '上映日期', '平均票房', '累積票房']

In [5]:
movie_box = pd.DataFrame(columns = column_name)

for i,row in enumerate(yearlist[1:]):
    movie = [j for j in row.stripped_strings]
    movie_box.loc[i] = movie

    # 若要逐列新增使用 loc
 
movie_box.head()

,排名,中文片名,英文片名,院數,映期,上映日期,平均票房,累積票房
0,1,玩命關頭8,Fast & Furious 8,25,85,2017/04/12,"7,341,639","183,540,980"
1,2,與神同行,Along with the Gods: The Two Worlds,24,60,2017/12/22,"7,075,516","169,812,385"
2,3,金牌特務：機密對決,Kingsman: The Golden Circle,25,88,2017/09/21,"5,059,644","126,491,105"
3,4,神力女超人,Wonder Woman,24,109,2017/05/30,"5,219,027","125,256,645"
4,5,雷神索爾3：諸神黃昏,Thor: Ragnarok,25,88,2017/10/24,"4,409,873","110,246,828"


In [6]:
# 票房的資料可以看到是以 str 儲存，為了方便之後的分析，要轉成數字型態 float or int
movie_box["平均票房"] = movie_box["平均票房"].str.replace(",", "").astype("float")
movie_box["累積票房"] = movie_box["累積票房"].str.replace(",", "").astype("float")

movie_box.head()

,排名,中文片名,英文片名,院數,映期,上映日期,平均票房,累積票房
0,1,玩命關頭8,Fast & Furious 8,25,85,2017/04/12,7341639.0,183540980.0
1,2,與神同行,Along with the Gods: The Two Worlds,24,60,2017/12/22,7075516.0,169812385.0
2,3,金牌特務：機密對決,Kingsman: The Golden Circle,25,88,2017/09/21,5059644.0,126491105.0
3,4,神力女超人,Wonder Woman,24,109,2017/05/30,5219027.0,125256645.0
4,5,雷神索爾3：諸神黃昏,Thor: Ragnarok,25,88,2017/10/24,4409873.0,110246828.0


In [7]:
movie_box.to_csv("movies_box_office_revised.csv", index = False, encoding="utf-8") # 儲存清洗後的資料